In [20]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [21]:
all_property = []

for page in range(28,29):
    print('Page: ', page)
    link = 'https://www.apartments.com/condos/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup)
#     page 1 - 5
#     results = soup.find_all('article', class_='placard placard-option has-header js-premiumplus')
#     page 5-28
#     results = soup.find_all('article', class_='placard placard-option no-header js-premium')

    # Loop through returned results and get the hyperlink of the properties of the page
#     print('results: ', results)
    for result in results:
#         print('result: ', result)
        find_tag = result.find('a', class_="property-link")['href']
        print(find_tag)
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
#     page 1 - 5
#         full_address = result.find('div', class_ = "property-information").text.strip().replace('\n', ' ')
#     page 5-28
        address = result.findAll('div', class_ = "js-url")
        full_address = ""
        for i in address:
            full_address = full_address + i.text + " "
        full_address = full_address.strip()
        print('full_address: ', full_address)
        city = new_results.find('div', class_ = "propertyAddressContainer").select('span')[1].text
#         print('city: ', city)
        state = new_results.find('div', class_ = "propertyAddressContainer").select('span')[3].text
#         print('state: ', state)
        zip_code = new_results.find('div', class_ = "propertyAddressContainer").select('span')[4].text
#         print('zip_code: ', zip_code)
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('div', class_="priceBedRangeInfoContainer")
#         print('house_details: ', house_details)
        bedroom_container = new_results.find('section', class_="availabilitySection")
#         print('bedroom_container: ', bedroom_container)
        all_the_rents = bedroom_container.findAll('div', class_="priceBedRangeInfo")
#         print('all_the_rents: ', all_the_rents)
        
        rent_obj = {}
        for rent_block in all_the_rents:
#             print('test: ', rent_block)
            price = rent_block.find('span', class_ = "rentLabel").text.strip()
            if "–" in price:
                price = price.split()[0]
            if "Call for Rent" in price or "/" in price:
                continue
            float_price = float(price.replace("$", "").replace(",", ""))
#             print('rent_block: ', rent_block)
#             print('price: ', price)
            testing = rent_block.find('h4', class_ = "detailsLabel").find('span', class_ = "leaseDepositLabel").text
            if "Not Available" in testing:
                continue
            bedroom_type = rent_block.find('h4', class_ = "detailsLabel").text.strip().split(",")[0]
            print(bedroom_type, " ", float_price)
            if bedroom_type in rent_obj and rent_obj[bedroom_type] < float_price:
                continue
            else:
                rent_obj[bedroom_type] = float_price
            bath_number = rent_block.find('h4', class_ = "detailsLabel").findAll('span')[2].text
#             print('bath_number: ', bath_number)
            house_square_feet = rent_block.find('h4', class_ = "detailsLabel").findAll('span')[3].text
            square_feet = house_square_feet.split(" ")[0];
            print('house_square_feet: ', house_square_feet)
#             lease_length = rent_block.find('span', class_="leaseDepositLabel").text.split(',')[0].strip()
#             print('lease_length: ', lease_length)
        
        rent_obj["City_Rent"] = city
        rent_obj["Type"] = "Condos"
        rent_obj["State_Rent"] = state
        rent_obj["Zip_code"] = zip_code
        rent_obj["Full_Address"] = full_address
        rent_obj["Rent_link"] = find_tag
        rent_obj["Bath_Info"] = bath_number
        rent_obj["Square_Feet_Data"] = house_square_feet
        rent_obj["Square_Feet"] = square_feet
#         rent_obj["Lease_length"] = lease_length
        
        all_property.append(rent_obj)
#         print(all_property)
        time.sleep(2)
        
# pprint(all_property);

Page:  28


In [18]:
rent_data = pd.DataFrame(all_property)
rent_data

,2 beds,City_Rent,Type,State_Rent,Zip_code,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,3 beds,4 beds,1 bed,Studio,5 beds
0,2825.0,Mountain View,Condos,CA,94040,"191 Showers Dr Mountain View, CA 94040",https://www.apartments.com/191-showers-dr-moun...,1 bath,675 sq ft,675,NaN,NaN,NaN,NaN,NaN
1,NaN,San Francisco,Condos,CA,94110,"3468 Mission St San Francisco, CA 94110",https://www.apartments.com/3468-mission-st-san...,2 baths,850 sq ft,850,3650.0,NaN,NaN,NaN,NaN
2,NaN,Santa Monica,Condos,CA,90403,"933 6th St Santa Monica, CA 90403",https://www.apartments.com/933-6th-st-santa-mo...,2 baths,"1,400 sq ft","1,400",5100.0,NaN,NaN,NaN,NaN
3,NaN,LOS ANGELES,Condos,CA,90035,"907 S SHENANDOAH St LOS ANGELES, CA 90035",https://www.apartments.com/907-s-shenandoah-st...,3 baths,"1,800 sq ft","1,800",NaN,6200.0,NaN,NaN,NaN
4,NaN,Oakland,Condos,CA,94607,"585 Ninth Street Oakland, CA 94607",https://www.apartments.com/585-ninth-street-oa...,1 bath,721 sq ft,721,NaN,NaN,2500.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4200.0,Beverly Hills,Condos,CA,90211,"8731 Clifton Way Beverly Hills, CA 90211",https://www.apartments.com/8731-clifton-way-be...,2 baths,"1,800 sq ft","1,800",NaN,NaN,NaN,NaN,NaN
296,2150.0,Glendale,Condos,CA,91202,"1331 N Brand Blvd Glendale, CA 91202",https://www.apartments.com/1331-n-brand-blvd-g...,1 bath,900 sq ft,900,NaN,NaN,NaN,NaN,NaN
297,NaN,Lawndale,Condos,CA,90260,"4559 W 159th St Lawndale, CA 90260",https://www.apartments.com/4559-w-159th-st-law...,1 bath,600 sq ft,600,NaN,NaN,1950.0,NaN,NaN
298,2900.0,Sunnyvale,Condos,CA,94085,"612 Arcadia Ter Sunnyvale, CA 94085",https://www.apartments.com/612-arcadia-ter-sun...,2 baths,"1,174 sq ft","1,174",NaN,NaN,NaN,NaN,NaN


In [19]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2021_Apartment.com_Condos_Data_page17-28.csv", index=False, header=True)